# LlamaCloud Demo - Ensemble Retrieval w/ Jira and Confluence

On this demo we will show how to use the LlamaCloud API to perform an ensemble retrieval from both Confluence and Jira.

The data source is previously setup on LlamaCloud and we will use the following data:


| ![image](./assets/confluence.png) | ![image](./assets/jira.png) |
|----------------------|----------------------|
| (Confluence)            | (Jira)            |

## Step 0: Setup environment config for LlamaCloud
Credentials can be get [here]("https://cloud.llamaindex.ai/").

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
import os

os.environ["LLAMA_CLOUD_API_KEY"] = "llx-"
os.environ["OPENAI_API_KEY"] = "sk-"

## Step 1: Initialize cloud indexes
We are defining one index per data source (Jira and Confluence) which will be used to retrieve from both.

In [3]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

In [33]:
confluence_index = LlamaCloudIndex(
    name='confluence',
    organization_id='49639377-3688-400a-ab41-54ff0f57a70d'
)

jira_index = LlamaCloudIndex(
    name='jira',
    organization_id='49639377-3688-400a-ab41-54ff0f57a70d'
)

vector_indices = [confluence_index, jira_index]

## Step 3: Define Ensemble Retriever
The goal here is to be able to retrieve from both indexes at same time.

In [34]:
from llama_index.core.selectors import PydanticMultiSelector
from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode

In [78]:
retriever_dict = {}
retriever_nodes = []

confluence_node = IndexNode(
    text=("Retrieves relevant context from the apple 10k documents"),
    index_id="confluence"
)

jira_node = IndexNode(
    text=("Retrieves relevant context from the apple 10k jira tasks"),
    index_id="jira"
)

retriever_nodes = [confluence_node, jira_node]
retriever_dict = {
   "confluence": confluence_index.as_retriever(
        similarity_top_k=15,
        rerank_top_n=15
    ),
   "jira": jira_index.as_retriever(
        similarity_top_k=15,
        rerank_top_n=15
    ),
}

## Step 4: Define Recursive Retriever

In [79]:
# the derived retriever will just retrieve all nodes
summary_index = SummaryIndex(retriever_nodes)

retriever = RecursiveRetriever(
    root_id="root",
    retriever_dict={"root": summary_index.as_retriever(), **retriever_dict},
)

## Testing it

In [80]:
nodes = await retriever.aretrieve(
    "What are the risk factors for Apple?"
)
print(f"Number of nodes: {len(nodes)}")
for node in nodes:
    print(node.node.get_text())

Number of nodes: 20
Risk Factors

The Company is exposed to the risk of write-downs on the value of its
inventory and other assets, in addition to purchase commitment cancellation
risk.

The Company records a write-down for product and component inventories that
have become obsolete or exceed anticipated demand, or for which cost exceeds
net realizable value. The Company also accrues necessary cancellation fee
reserves for orders of excess products and components. The Company reviews
long-lived assets, including capital assets held at its suppliers’ facilities
and inventory prepayments, for impairment whenever events or circumstances
indicate the assets may not be recoverable. If the Company determines that an
impairment has occurred, it records a write-down equal to the amount by which
the carrying value of the asset exceeds its fair value. Although the Company
believes its inventory, capital assets, inventory prepayments and other assets
and purchase commitments are currently recover

## Step 5: Define the Reranker and RetrieverQueryEngine
Given that we are retrieving from both indexes now we setup a reranker + a retriever query engine.

In [ ]:
%pip install llama-index-postprocessor-cohere-rerank
%pip install matplotlib

In [ ]:
os.environ["COHERE_API_KEY"] = ""

In [111]:
from llama_index.core.postprocessor import LLMRerank, SentenceTransformerRerank
from llama_index.postprocessor.cohere_rerank import CohereRerank

# reranker = LLMRerank()
# reranker = SentenceTransformerRerank(top_n=10)
reranker = CohereRerank(top_n=20)

In [112]:
# define RetrieverQueryEngine
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever, node_postprocessors=[reranker])

## Query it

In [ ]:
from llama_index.core.response.notebook_utils import display_response

In [113]:
response = query_engine.query(
    "What are the risk factors for Apple 2021? also include in the final answer: who's assigned to analyze it and task description"
)

In [115]:
display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

**`Final Response:`** The risk factors for Apple in 2021 include challenges related to business risks, macroeconomic and industry risks, risks associated with foreign currency exchange rates, political events, trade disputes, natural disasters, public health issues, and industrial accidents. These factors could impact the company's results of operations and financial condition. Emanuel Ferreira is assigned to analyze the risk factors for Apple 2021 as part of the task titled "Analyze Apple Risk Factors."

---

**`Source Node 1/20`**

**Node ID:** dd60d844-f467-4ab5-9018-309d49c877b4<br>**Similarity:** 0.9978342<br>**Text:** These and other factors could have a materially
adverse impact on the Company’s results of operations and financial condition.

The Company has historically experienced higher net sales in its first quarter
compared to other quarters in its fiscal year due in part to seasonal holiday
demand. Additionally, new product and service introductions can significantly
impact net sales, cost of sales, and operating expenses. Further, the Company
generates a significant portion of its net sales from a ...<br>**Metadata:** {'title': 'Apple 2021', 'page_id': '393217', 'status': 'current', 'url': 'https://ext-runllama.atlassian.net//spaces/~71202027022399ab9743fe859a981d4daa4245/pages/393217/Apple+2021', 'pipeline_id': '74ada4b3-17ca-4126-87b7-6446ce5170a7', 'document_9_page_label': 1, 'Header_1': 'Business Risks', 'Header_2': 'Apple Inc. | 2021 Form 10-K | 14'}<br>

---

...

**`Source Node 19/20`**

**Node ID:** d5faaa76-b553-42e4-a92e-64cdedc40a54<br>**Similarity:** 0.524075<br>**Text:** Gather Historical Stock Data for Appl 
 None<br>**Metadata:** {'id': '10001', 'title': 'Gather Historical Stock Data for Appl', 'url': 'https://ext-runllama.atlassian.net/browse/KAN-2', 'created_at': '2024-09-04T08:49:59.976-0300', 'updated_at': '2024-09-04T08:51:57.465-0300', 'labels': [], 'status': 'Done', 'assignee': 'Emanuel Ferreira', 'reporter': 'Emanuel Ferreira', 'project': 'apple-10k-project', 'issue_type': 'Task', 'priority': 'Medium', 'epic_key': 'KAN-1', 'epic_summary': 'Apple Stock Price Analysis and Apple-Tesla Risk Assessment', 'epic_description': '', 'pipeline_id': 'c025bc86-aa1a-4171-b8dc-5d0db9cb4628'}<br>

---

**`Source Node 20/20`**

**Node ID:** 45a64763-f93c-4381-9f5b-2d3281bfbc32<br>**Similarity:** 0.4414322<br>**Text:** Apple Stock Price Analysis and Apple-Tesla Risk Assessment 
 Collect historical stock price data for Apple over the last 5 years, including daily closing prices, volume, and any relevant financial indicators.<br>**Metadata:** {'id': '10000', 'title': 'Apple Stock Price Analysis and Apple-Tesla Risk Assessment', 'url': 'https://ext-runllama.atlassian.net/browse/KAN-1', 'created_at': '2024-09-04T08:48:27.604-0300', 'updated_at': '2024-09-04T08:49:30.823-0300', 'labels': [], 'status': 'To Do', 'assignee': 'Emanuel Ferreira', 'reporter': 'Emanuel Ferreira', 'project': 'apple-10k-project', 'issue_type': 'Epic', 'priority': 'Medium', 'epic_key': 'KAN-1', 'epic_summary': 'Apple Stock Price Analysis and Apple-Tesla Risk Assessment', 'epic_description': '', 'pipeline_id': 'c025bc86-aa1a-4171-b8dc-5d0db9cb4628'}<br>

In [119]:
response = query_engine.query(
    "Which tasks Emanuel Ferreira is working on? give details about the task and the id"
)

In [120]:
print(str(response))

Emanuel Ferreira is working on the following tasks:
1. Analyze Apple Risk Factors (Task ID: 10003) - Identify and document risks related to Apple such as market volatility, competition, supply chain issues, and regulatory challenges.
2. Perform Apple Stock Price Trend Analysis (Task ID: 10002) - Analyze Apple’s stock price trends, including identifying key support and resistance levels, trend lines, and significant price movements.
